In [ ]:
#  to send a mail
import smtplib
# to use aes and des inbuilt function function
from Crypto.Cipher import DES
from Crypto.Random import get_random_bytes
#  to read the mail content
import imaplib
import email
from email.header import decode_header
#  to use the function sys.exit ie to finish the program
import sys

# ceasor cipher encryption

def caesar_cipher_encrypt(text, shift):
    encrypted_text = ""

    for char in text:
        if char.isalpha():
            # Determine whether the character is uppercase or lowercase
            is_upper = char.isupper()

            # Shift the character while preserving case
            char_code = ord(char)
            shifted_code = char_code + shift
            if is_upper:
                if shifted_code > ord('Z'):
                    shifted_code -= 26
                elif shifted_code < ord('A'):
                    shifted_code += 26
            else:
                if shifted_code > ord('z'):
                    shifted_code -= 26
                elif shifted_code < ord('a'):
                    shifted_code += 26

            encrypted_char = chr(shifted_code)
        else:
            # Keep non-alphabetic characters unchanged
            encrypted_char = char

        encrypted_text += encrypted_char

    return encrypted_text


# ceasor cipher decryption

def caesar_cipher_decrypt(encrypted_text, shift):
    decrypted_text = caesar_cipher_encrypt(encrypted_text, -shift)
    return decrypted_text



# mono alphabatic cipher

def create_monoalphabetic_cipher(key):
    if len(key) != 26:
        raise ValueError("Key must be a permutation of the alphabet (26 characters).")

    cipher = {}
    for i in range(26):
        cipher[chr(ord('a') + i)] = key[i]
        cipher[chr(ord('A') + i)] = key[i]

    return cipher

# monoalphabatic ciper encrytion

def monoalphabetic_encrypt(plain_text, key):
    cipher = create_monoalphabetic_cipher(key)
    encrypted_text = "".join(cipher[char] if char in cipher else char for char in plain_text)
    return encrypted_text

# monoalphabatic ciper decryption

def monoalphabetic_decrypt(encrypted_text, key):
    cipher = create_monoalphabetic_cipher(key)
    reversed_cipher = {v: k for k, v in cipher.items()}
    decrypted_text = "".join(reversed_cipher[char] if char in reversed_cipher else char for char in encrypted_text)
    return decrypted_text



# playfair ciper

def preprocess_text(plain_text):
    # Remove spaces and convert to uppercase
    plain_text = plain_text.replace(" ", "").upper()
    # Replace 'J' with 'I' to make it a 5x5 matrix
    plain_text = plain_text.replace("J", "I")
    # Add an 'X' at the end for even pairing
    if len(plain_text) % 2 == 1:
        plain_text += "X"
    return plain_text

def create_playfair_matrix(key):
    key = key.replace(" ", "").upper()
    key = key.replace("J", "I")
    key_set = set(key)
    alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ"

    for char in key_set:
        alphabet = alphabet.replace(char, "")

    matrix = list(key)
    for char in alphabet:
        matrix.append(char)

    playfair_matrix = [matrix[i:i + 5] for i in range(0, 25, 5)]
    return playfair_matrix

def find_char_positions(matrix, char):
    positions = []
    for i in range(5):
        for j in range(5):
            if matrix[i][j] == char:
                positions.append((i, j))
    return positions


# playfair encryption

def playfair_encrypt(plain_text, key):
    plain_text = preprocess_text(plain_text)
    playfair_matrix = create_playfair_matrix(key)
    encrypted_text = ""

    for i in range(0, len(plain_text), 2):
        char1, char2 = plain_text[i], plain_text[i + 1]
        positions1 = find_char_positions(playfair_matrix, char1)
        positions2 = find_char_positions(playfair_matrix, char2)

        if len(positions1) == 0 or len(positions2) == 0:
            raise ValueError(f"Character not found in the matrix: {char1} or {char2}")

        row1, col1 = positions1[0]
        row2, col2 = positions2[0]

        if row1 == row2:
            encrypted_char1 = playfair_matrix[row1][(col1 + 1) % 5]
            encrypted_char2 = playfair_matrix[row2][(col2 + 1) % 5]
        elif col1 == col2:
            encrypted_char1 = playfair_matrix[(row1 + 1) % 5][col1]
            encrypted_char2 = playfair_matrix[(row2 + 1) % 5][col2]
        else:
            encrypted_char1 = playfair_matrix[row1][col2]
            encrypted_char2 = playfair_matrix[row2][col1]

        encrypted_text += encrypted_char1 + encrypted_char2

    return encrypted_text

# playfair decryption

def playfair_decrypt(encrypted_text, key):
    playfair_matrix = create_playfair_matrix(key)
    decrypted_text = ""

    for i in range(0, len(encrypted_text), 2):
        char1, char2 = encrypted_text[i], encrypted_text[i + 1]
        positions1 = find_char_positions(playfair_matrix, char1)
        positions2 = find_char_positions(playfair_matrix, char2)

        if len(positions1) == 0 or len(positions2) == 0:
            raise ValueError(f"Character not found in the matrix: {char1} or {char2}")

        row1, col1 = positions1[0]
        row2, col2 = positions2[0]

        if row1 == row2:
            decrypted_char1 = playfair_matrix[row1][(col1 - 1) % 5]
            decrypted_char2 = playfair_matrix[row2][(col2 - 1) % 5]
        elif col1 == col2:
            decrypted_char1 = playfair_matrix[(row1 - 1) % 5][col1]
            decrypted_char2 = playfair_matrix[(row2 - 1) % 5][col2]
        else:
            decrypted_char1 = playfair_matrix[row1][col2]
            decrypted_char2 = playfair_matrix[row2][col1]

        decrypted_text += decrypted_char1 + decrypted_char2

    return decrypted_text











#polyalphabtic cipher

def polyalphabetic_encrypt(plain_text, key):
    encrypted_text = ""
    key_len = len(key)

    for i, char in enumerate(plain_text):
        if char.isalpha():
            shift = ord(key[i % key_len].lower()) - ord('a')
            if char.islower():
                encrypted_char = chr(((ord(char) - ord('a') + shift) % 26) + ord('a'))
            else:
                encrypted_char = chr(((ord(char) - ord('A') + shift) % 26) + ord('A'))
            encrypted_text += encrypted_char
        else:
            encrypted_text += char

    return encrypted_text

def polyalphabetic_decrypt(encrypted_text, key):
    decrypted_text = ""
    key_len = len(key)

    for i, char in enumerate(encrypted_text):
        if char.isalpha():
            shift = ord(key[i % key_len].lower()) - ord('a')
            if char.islower():
                decrypted_char = chr(((ord(char) - ord('a') - shift) % 26) + ord('a'))
            else:
                decrypted_char = chr(((ord(char) - ord('A') - shift) % 26) + ord('A'))
            decrypted_text += decrypted_char
        else:
            decrypted_text += char

    return decrypted_text


# AES

from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# AES encryption
def aes_encrypt(key, plaintext):
    cipher = AES.new(key, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(plaintext.encode())
    return ciphertext, cipher.nonce

# AES decryption
def aes_decrypt(key, ciphertext, nonce):
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    decrypted_text = cipher.decrypt(ciphertext).decode()
    return decrypted_text






encrypted_text=""


print("\t\t\t Welcome to email encryption system \n")
print(" Press (1) to send the mail with encrypted text \n Press (2) to read the mail ")
send_recieve = int(input())

if(send_recieve==1):


  # email = "temp48@gmail.com"    Please use your own email
  # reciever_email = "temp48@gmail.com"  Please use your own email
  subject = "Testing"

#  to open at the time of submission

  # reciever_email = input("Reciever's Mail : ")
  # subject = input("Subject :")


  print("Select the method of encryption \n")
  print("(1) ceasor cipher")
  print("(2) Monoalphabtic cipher")
  print("(3) playfair cipher")
  print("(4) polyalphabatic cipher")
  print("(5) DES")
  print("(6) AES")






#caesor cipher

  encryption_technique = int(input("choose one among encrytpion technique : "))
  if(encryption_technique==1):
    print("enter the content of the mail you want to send ")
    content = input()
    print("enter the amount of shifting \t note by default shifting is by 3 ")
    shift_amount = int(input())
    encrypted_text = caesar_cipher_encrypt(content, shift_amount)
    print("Encrypted Text:", encrypted_text)

# monoalphabatic cipher

  elif(encryption_technique==2):
    print("enter the content of the mail you want to send \n")
    content = input()

    print("Do you want to create our own key or use inbuilt key \n")
    print("press 1) For default key \n      2)  For creating own key")
    default_key_or_self_key_monoalphabatic = int(input())
    monoalphabatic_key=""

    if(default_key_or_self_key_monoalphabatic ==1):
      monoalphabatic_key = "QWERTYUIOPASDFGHJKLZXCVBNM"
      print("your key to decrypt is  : ", monoalphabatic_key)
    else:
      print("Note key must be a permutation of all alphabets i.e 26 different characters ")
      monoalphabatic_key = input()

    encrypted_text = monoalphabetic_encrypt(content, monoalphabatic_key)
    print("Encrypted Text:", encrypted_text)

#playfiar cipher

  elif(encryption_technique==3):
    print("enter the content of the mail you want to send \n")
    print("Note : In this technique the content should only be alphabets\n")
    content = input()

    print("Do you want to create our own key or use inbuilt key \n")
    print("press 1) For default key \n      2)  For creating own key")
    default_key_or_self_key_playfiar = int(input())
    playfiar_key=""

    if(default_key_or_self_key_playfiar ==1):
      playfiar_key = "Jiitrocks"
      print("your key to decrypt is :  ",playfiar_key)
    else:
      playfiar_key = input()
    encrypted_text = playfair_encrypt(content, playfiar_key)
    print("Encrypted Text:", encrypted_text)

#polyalphabatic cipher
  elif(encryption_technique==4):
    print("enter the content of the mail you want to send \n")
    content = input()

    print("Do you want to create our own key or use inbuilt key \n")
    print("press 1) For default key \n      2)  For creating own key")
    default_key_or_self_key_polyalphabatic = int(input())
    polyalphabatic_key=""

    if(default_key_or_self_key_polyalphabatic ==1):
      polyalphabatic_key +="Jiitrocks"
      print("your key to decrypt is : ",polyalphabatic_key)
    else:
      polyalphabatic_key = input()
    encrypted_text = polyalphabetic_encrypt(content, polyalphabatic_key)
    print("Encrypted Text:", encrypted_text)


  elif(encryption_technique==5):
    print("enter the content of the mail you want to send \n")
    content = input()

    # Make sure the message is a multiple of 8 bytes
    while len(content) % 8 != 0:
      content += " "  # Padding with spaces
    # Encrypt the message

    print("Do you want to create our own key or use inbuilt key \n")
    print("press 1) For default key \n      2)  For creating own key")
    default_key_or_self_key_des = int(input())
    des_key=""

    if(default_key_or_self_key_des ==1):
      # Generate a random 8-byte (64-bit) DES key
      des_key = get_random_bytes(8)
      print("this s key : ", des_key)
    else:
      print("Enter the key in the form of 8 bytes")
      des_key = input()

    # Create a DES cipher object using the key and the DES MODE_ECB mode
    cipher = DES.new(des_key, DES.MODE_ECB)
    # Encrypt the message
    encrypted_text = cipher.encrypt(content.encode())


  elif(encryption_technique==6):
    print("enter the content of the mail you want to send \n")
    content = input()
    aes_key = get_random_bytes(32)
    print(aes_key)

    # Encrypt the plaintext
    encrypted_text, nonce = aes_encrypt(aes_key, content)

  else:
    print("choose the statement correctly next time :")
    sys.exit()

  text = f"Subject : {subject}\n\n{encrypted_text}"
  server = smtplib.SMTP("smtp.gmail.com",587)
  server.starttls()
  server.login(email,"gkpx rhxh vlta tvxo")
  server.sendmail(email,reciever_email,text)
  print("Email has been sent to : ",reciever_email)


elif(send_recieve==2):

        # Set your Gmail credentials
    # email_address = "temp48@gmail.com"    # Please use your own email
    # password = ""   Please use your own password

    # Connect to the Gmail IMAP server
    imap_server = imaplib.IMAP4_SSL("imap.gmail.com")
    imap_server.login(email_address, password)

    # Select the mailbox you want to read emails from (e.g., "inbox")
    mailbox = "inbox"
    imap_server.select(mailbox)

    # Search for emails based on some criteria (e.g., all emails)
    status, email_ids = imap_server.search(None, "ALL")

    # Get a list of email IDs
    email_id_list = email_ids[0].split()

    # Fetch the latest email (you can specify a different email ID)
    latest_email_id = email_id_list[-1]

    # Fetch the email content
    status, email_data = imap_server.fetch(latest_email_id, "(RFC822)")

    # Parse the email
    raw_email = email_data[0][1]
    message = email.message_from_bytes(raw_email)

    subject = message.get("Subject", "No Subject")
    subject, encoding = decode_header(subject)[0]
    if isinstance(subject, bytes):
        subject = subject.decode(encoding or "utf-8")
    print(f"Subject: {subject}")
    print(f"From: {message['From']}")
    print(f"To: {message['To']}")
    print(f"Date: {message['Date']}")

    # Print the email body (plain text content)
    for part in message.walk():
        if part.get_content_type() == "text/plain":
            email_body = part.get_payload(decode=True).decode("utf-8")
            print("Email Body:")
            print(email_body)

    # Logout from the IMAP server

    imap_server.logout()
    encrypted_text = email_body
    print("\n\n Do you want to decrypt the text in the mail ")
    print("press (1) For decrypt \n Press (2) to end")
    wish_decrypt = int(input())
    if(wish_decrypt==1):

      print("Select the method of decryption \n")
      print("(1) ceasor cipher")
      print("(2) Monoalphabtic cipher")
      print("(3) playfair cipher")
      print("(4) polyalphabatic cipher")
      print("(5) DES")
      print("(6) AES")
      decrypted_text=""

      decryption_technique = int(input("choose one among decryption technique : "))
      print("Enter the key to encrypt the text")
      decrypt_key = input()
      if(decryption_technique==1):
        decrypted_text += caesar_cipher_decrypt(encrypted_text, int(decrypt_key))

      elif(decryption_technique==2):
        decrypted_text += monoalphabetic_decrypt(encrypted_text, decrypt_key)

      elif(decryption_technique==3):
        decrypted_text += playfair_decrypt(encrypted_text, decrypt_key)

      elif(decryption_technique==4):
        decrypted_text += polyalphabetic_decrypt(encrypted_text, decrypt_key)

      elif(decryption_technique==5):
        decrypted_message += cipher.decrypt(encrypted_message)

      elif(decryption_technique==6):
        decrypted_text += aes_decrypt(decrypt_key, ciphertext, nonce)

      else:
        print("Choose a valid choice")
      print(decrypted_text)
    else:
      sys.exit()

else:
  print("select the correct option \n")
  sys.exit()

